In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import streamlit as st
import tensorflow as tf
import pickle as pkl
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import  Pipeline
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense
from tensorflow.keras.callbacks import EarlyStopping

In [61]:
##reading Datasets 
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [62]:
#Dropping the columns

data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [63]:
#Converting Categorical Feature  --- Gender
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])


In [64]:
#Converting Categorical Feature--- Geography

onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [65]:
##Concating the onehot encoded Feature 
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [66]:
##Independent and Dependent Feature 
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [67]:
##train_test_split Dependent and Independent Feature
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [68]:
##Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [69]:
##Save the encoders and scaler

with open('onehot_encoder_geo.pkl','wb') as file :
    pkl.dump(onehot_encoder_geo,file)

with open('label_encoder_gender.pkl','wb') as file :
    pkl.dump(label_encoder_gender,file)

with open('scaler.pkl','wb') as file:
    pkl.dump(scaler,file)

In [86]:
##Define a function to create the model and try differne Parameter(Keras Regression)

def create_models(neurons=32,layers=1):
    models =Sequential()
    models.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        models.add(Dense(neurons,activation='relu'))

    models.add(Dense(1))
    models.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

    return models

In [87]:
##Create a Keras Regressor 
models = KerasRegressor(neurons=32,layers=1,build_fn=create_models,epochs=100,batch_size=10,verbose=0)


In [88]:
##Define the gridsearch parameters 

param_grid = {
    'neurons':[16,32,64,128],
    'layers':[1,2,3,4],
    'epochs':[50,100]
    }

In [90]:
##Perform grid Search
grid = GridSearchCV(estimator=models,param_grid=param_grid,n_jobs=1,cv=3)

grid_result = grid.fit(X_train,y_train)

print("Best : %f using %s" % (grid_result.best_score_, grid_result.best_params_))

d:\DeepLearning\CustomerChurn_Regression\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\DeepLearning\CustomerChurn_Regression\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\DeepLearning\CustomerChurn_Regression\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\DeepLearning\CustomerChurn_Regression\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pa

KeyboardInterrupt: 

In [37]:
estimator.get_params().keys()

NameError: name 'estimator' is not defined